In [34]:
import pybullet
import gym
import numpy as np
import matplotlib.pyplot as plt
import pybullet_data
import math
import rrtstar
import mpc
import time
import torch
from environment import Environment
from maddpg import MADDPGAgentTrainer
import tensorflow as tf
import tensorflow.keras.layers as layers

In [ ]:
#!git clone https://github.com/ROBOTIS-GIT/turtlebot3.git

In [ ]:
# RRT
goal = (5.25, 3.5, 0)
start = (0.25, 0.25, 0)
samplingArea = [0, 10, 0, 5]
RRT = rrtstar.RRT(start, goal, [], samplingArea, 'rrtstar', 'rectangle', 2)
path = RRT.planning()

In [36]:
print(path)
path = np.flip(path, 0)
print(path)


[[5.25, 3.5, 0], [5.25, 3.5, 0], [1.5829907845515867, 0.9889830048903997, 0.8847724379636436], [0.25, 0.25, 0]]
[[0.25       0.25       0.        ]
 [1.58299078 0.988983   0.88477244]
 [5.25       3.5        0.        ]
 [5.25       3.5        0.        ]]


In [38]:
extended_path = []
for i in range(len(path)):
    print(path[i])
for i in range(len(path)):
    if i + 1 != len(path) - 1:
        angle = 0.39
        path_part = list(zip(np.linspace(path[i][0], path[i+1][0], 100), np.linspace(path[i][1], path[i+1][1], 100)))
        for part in path_part:
            extended_path.append((part[0], part[1], angle))
    else:
        break
        
print(extended_path)

[0.25 0.25 0.  ]
[1.58299078 0.988983   0.88477244]
[5.25 3.5  0.  ]
[5.25 3.5  0.  ]
[(0.25, 0.25, 0.39), (0.26346455337930896, 0.25746447479687273, 0.39), (0.2769291067586179, 0.26492894959374547, 0.39), (0.2903936601379269, 0.2723934243906182, 0.39), (0.30385821351723585, 0.2798578991874909, 0.39), (0.3173227668965448, 0.2873223739843636, 0.39), (0.3307873202758537, 0.29478684878123634, 0.39), (0.3442518736551627, 0.3022513235781091, 0.39), (0.35771642703447165, 0.3097157983749818, 0.39), (0.3711809804137806, 0.3171802731718545, 0.39), (0.3846455337930896, 0.3246447479687272, 0.39), (0.3981100871723985, 0.33210922276559995, 0.39), (0.41157464055170745, 0.3395736975624727, 0.39), (0.4250391939310164, 0.3470381723593454, 0.39), (0.4385037473103254, 0.35450264715621815, 0.39), (0.45196830068963434, 0.3619671219530909, 0.39), (0.4654328540689433, 0.3694315967499636, 0.39), (0.47889740744825227, 0.37689607154683635, 0.39), (0.49236196082756123, 0.38436054634370903, 0.39), (0.505826514206

In [41]:
physicsClient = pybullet.connect(pybullet.GUI)
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
pybullet.setGravity(0,0,-9.8)

# Loading chessgrid plane
planeID = pybullet.loadURDF('plane.urdf')
chess_grid = pybullet.loadTexture('checker_huge.gif')
pybullet.changeVisualShape(planeID, -1, textureUniqueId=chess_grid)

# Setting up overhead camera
viewMatrix = pybullet.computeViewMatrix(
    cameraEyePosition=[0.75,0.25,3],
    cameraTargetPosition=[0.75,0.25,0],
    cameraUpVector=[0,1,0])

projectionMatrix = pybullet.computeProjectionMatrixFOV(
    fov=45.0,
    aspect=1.0,
    nearVal=0.1,
    farVal=3.1)

width, height, rgbImg, depthImg, segImg = pybullet.getCameraImage(width = 224, height = 224, viewMatrix=viewMatrix, projectionMatrix = projectionMatrix)
player = pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', start, pybullet.getQuaternionFromEuler([0,0,0]))
pybullet.changeVisualShape(player, 0, rgbaColor=[0,0,1,1])

pybullet.setRealTimeSimulation(1)

In [42]:
def applyAction(actions, orig_state):
    new_states = []
    for i in range(G):
        action_set = actions[i]
        for j in range(H):
            action = action_set[j]
            action_3d = [fM * action[0], fM * action[1], 0]
            for x in range(10):
                pybullet.stepSimulation()
                time.sleep(1./240.)
                current_pos = pybullet.getBasePositionAndOrientation(player)[0]
                #pybullet.applyExternalForce(player, -1, action_3d, current_pos, pybullet.WORLD_FRAME)
                pybullet.resetBaseVelocity(player, action_3d)
            pybullet.resetBaseVelocity(player, [0,0,0])

        
        new_states.append(pybullet.getBasePositionAndOrientation(player))
        pybullet.resetBasePositionAndOrientation(player, orig_state[0], orig_state[1])
    return new_states
        

In [43]:
def getCost(states, target):
    #target = [goal[0], goal[1], 0]
    target_pos = [target[0], target[1]]
    target_angle = target[2]
    costs = []
    for i in range(G):
        current = states[i]
        current_pos = current[0]
        current_angle = pybullet.getEulerFromQuaternion(current[1])[2]
        # Double check above. Based on angle used for initial setting position
        cost = math.sqrt(((current_pos[0] - target_pos[0]) ** 2) + ((current_pos[1] - target_pos[1]) ** 2))
        cost += abs(current_angle - target_angle)
        costs.append(cost)
    return costs

In [44]:
H = 5
G = 10
T = 5
K = 1
fM = 1

def cem(path):
    pybullet.setRealTimeSimulation(0)
    total_actions = []
    for i in range(len(path) - 1):
    #for i in range(1000):
        initial_actions = np.random.normal(0,1,(G,H,2))
        state = pybullet.getBasePositionAndOrientation(player)
        print(state)
        for t in range(T):
            new_states = applyAction(initial_actions, state)
            costs = getCost(new_states, path[i + 1])
            #costs = getCost(new_states, path[1])
            sorted_actions = [act for _,act in sorted(zip(costs, initial_actions), key=lambda pair: pair[0])]
            new_mean = np.mean(sorted_actions[:K])
            new_std = np.std(sorted_actions[:K])
            new_actions = np.random.normal(new_mean, new_std, (G,H,2))
            #print(sorted_actions[:K])
            #print(new_actions)
            initial_actions = np.concatenate((sorted_actions[:K], new_actions))
            
        best_action = initial_actions[0][0] # Because no horizon length
        action = [fM * best_action[0], fM * best_action[1], 0]
        total_actions.append(action)
        #print(action)
        for x in range(10):
            pybullet.stepSimulation()
            time.sleep(1./240.)
            #pybullet.applyExternalForce(player, -1, action, state[0], pybullet.WORLD_FRAME)
            pybullet.resetBaseVelocity(player, action)
        pybullet.resetBaseVelocity(player, [0,0,0])
    return total_actions

            
            


In [45]:
actions = cem(extended_path)
#pybullet.setRealTimeSimulation(1)

((0.24547763901328334, 0.2499992203260546, 0.0014276234272820277), (-5.281990763157658e-05, 0.08223610791486893, 0.0010457326066936055, 0.9966123249330135))
((0.23484558364622357, 0.22783398454698114, 0.0014129235411449936), (-0.00019680694876383459, 0.08210535487780975, 0.0026009715554045055, 0.9966202420753764))
((0.2801553776542588, 0.2108262596929758, 0.001422718561886136), (-4.952870357483697e-05, 0.08216231717338211, 0.000727213045879842, 0.9966186945591549))
((0.31367185251505464, 0.21300166793226985, 0.0016037436738222867), (-6.816568696042683e-05, 0.08345539172676826, 0.0008027872560577177, 0.9965111883354809))
((0.35648849723834897, 0.2183893983301419, 0.001457533639789444), (-3.381748465842019e-05, 0.0824074126572942, 0.00042946686258749805, 0.9965986317237858))
((0.3665542641963982, 0.2723352287335422, 0.0014341829342281386), (-0.0001378483323493555, 0.08223221973119851, 0.0016313817509832592, 0.9966118510380557))
((0.3183012022308891, 0.27081379484222395, 0.001404205289296

In [46]:
print(actions)

[[-0.28539061265746846, -0.6105496358751599, 0], [1.2224032427184957, -0.468005369760023, 0], [0.9171168892190011, 0.061368399580581545, 0], [1.1603849146613912, 0.14910903356607805, 0], [0.27001563290073166, 1.4597004958568955, 0], [-1.296310645109806, -0.05602088232486925, 0], [0.48956868027341427, -0.6349250709893872, 0], [0.27301564917015214, 0.4432516847324944, 0], [-0.2471992884401517, 2.7556099175772806, 0], [0.7272141233944786, -1.073382783967635, 0], [-0.13775783982783518, -0.1617748830945276, 0], [1.6650352883163744, 0.9093851545784255, 0], [0.0230127424582751, 0.14080214421106677, 0], [-0.7265667183141262, -0.3273553020555622, 0], [-0.0306801671069315, 0.003027408211156324, 0], [-0.8633469524767314, 0.7508960857663782, 0], [0.5993307120079374, 1.0841215535557045, 0], [0.8721625644165993, 1.3157112848265373, 0], [0.24418277402375704, -0.9464785516437281, 0], [3.4281651644905375, -0.0806334909017804, 0], [0.2295173216958744, -2.047550071181235, 0], [0.05777586703689441, 1.1680

In [48]:
pybullet.resetBasePositionAndOrientation(player, [0.25,0.25,0], pybullet.getQuaternionFromEuler([0,0,0]))
for action in actions:
    for x in range(10):
        pybullet.stepSimulation()
        time.sleep(1./240.)
        #pybullet.applyExternalForce(player, -1, action, state[0], pybullet.WORLD_FRAME)
        pybullet.resetBaseVelocity(player, action)
    pybullet.resetBaseVelocity(player, [0,0,0])

In [40]:
pybullet.disconnect()

error: Not connected to physics server.